# **I. READING SATELLITE IMAGES FROM SENTINEL-2 & LandSat-8**

Reading data from these images are critical due to:
- Analysis of Surface material & albedo of Road and Pavement: specifically ustilize LandSat 8's thermal infrared (TIRS - Band 10 & Band 11) Shortwave Infrared (SWIR1 - SWIR2). There are scientific proven albedo formula using Landsat Band.
- Analysis of Vegetation areas: utilize Red Edge Band (B5 -> B8A.) from Sentinel-2 satellite

## **1. Define geo-boundings & time**

In [4]:
# Define bounding box (latitude, longitude) for area of interest - Bronx & Manhattan of New York
lower_left = (40.75, -74.01)     # (lat, lon)
upper_right = (40.88, -73.86)

# Calculate bounds as (min_lon, min_lat, max_lon, max_lat)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

# Define your time period of interest
time_window = "2021-06-01/2021-09-01"

## **2. Connect to Sentinel-2 & Landsat-8**

In [8]:
import warnings
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import RdYlGn, jet, RdBu

import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

# Connect to the Microsoft Planetary Computer STAC endpoint
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

# Search for Sentinel-2 scenes (with cloud cover < 30%)
sentinel_search = stac.search(
    bbox=bounds, 
    datetime=time_window,
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 30}},
)
sentinel_items = list(sentinel_search.get_items())
print('Number of Sentinel-2 scenes found:', len(sentinel_items))

# Search for Landsat scenes (using a similar cloud cover threshold, adjust as needed)
landsat_search = stac.search(
    bbox=bounds,
    datetime=time_window,
    collections=["landsat-8-c2-l2"],
    query={"eo:cloud_cover": {"lt": 20}},
)
landsat_items = list(landsat_search.get_items())
print('Number of Landsat scenes found:', len(landsat_items))


Number of Sentinel-2 scenes found: 10
Number of Landsat scenes found: 5


## **3. Loading images from Sentinel-2 & Landsat 8**

In [ ]:
signed_sentinel_items = [planetary_computer.sign(sentinel_items).to_dict() for sentinel_item in sentinel_items]

resolution = 10
scale = resolution / 111320

sentinel_data = stac_load(
    sentinel_items,
    bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
    crs="EPSG:4326",  # Latitude-Longitude coordinate system
    resolution=scale, 
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)
median_s2 = sentinel_data.median(dim="time").compute()

In [ ]:
# Sign Landsat items
signed_landsat_items = [planetary_computer.sign(item).to_dict() for item in landsat_items]

landsat_data = stac_load(
    landsat_items,
    bands=["B2", "B3", "B4", "B5", "B6", "B7"],
    crs="EPSG:4326",
    resolution=scale,
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

# Create a median composite for Landsat
median_ls = landsat_data.median(dim="time").compute()
